# KNN Classification


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        # Compute distances between x and all examples in the training set
        distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]
        # Sort by distance and return indices of the first k neighbors
        k_indices = np.argsort(distances)[:self.k]
        # Extract the labels of the k nearest neighbor training samples
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Return the most common class label
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

In [ ]:
df = pd.read_csv('iris.csv')
df['variety'].replace({'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}, inplace=True)

<ipython-input-3-22cf187c84df>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['variety'].replace({'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}, inplace=True)
<ipython-input-3-22cf187c84df>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['variety'].replace({'Setosa': 0, 'Versicolor': 1, 'Vi

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1].values, df.iloc[:, -1].values, test_size=0.2, random_state=42)

In [ ]:
model = KNN()
model.fit(X_train, y_train)

In [ ]:
predicted_classes = model.predict(X_test)
actual_classes = y_test
print(predicted_classes, actual_classes)
correct = len([i for i in range(len(predicted_classes)) if predicted_classes[i] == actual_classes[i]])
accuracy = correct / len(predicted_classes)
print("Accuracy:", accuracy*100)


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0] [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy: 100.0


# SVM Classification

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Linear kernel function
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

# Support Vector Machine (SVM) class
class SVM:
    def __init__(self, C=1, max_iter=1000):
        self.C = C  # Regularization parameter
        self.max_iter = max_iter
        self.alpha = None
        self.b = None
        self.kernel = linear_kernel
        self.X_train = None
        self.y_train = None

    # Fit the SVM model (train the model)
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.X_train = X
        self.y_train = y

        # One-vs-rest strategy: Train a separate SVM for each class
        self.models = []
        self.classes = np.unique(y)

        for class_label in self.classes:
            y_binary = np.where(y == class_label, 1, -1)  # Class vs all other classes
            model = self._train_svm(X, y_binary)
            self.models.append(model)

    # Train a binary SVM for one class vs all others
    def _train_svm(self, X, y):
        n_samples, n_features = X.shape
        alpha = np.zeros(n_samples)
        b = 0

        for _ in range(self.max_iter):
            alpha_prev = np.copy(alpha)
            for i in range(n_samples):
                j = np.random.choice([x for x in range(n_samples) if x != i])

                E_i = self._compute_error(X, y, i, alpha, b)
                E_j = self._compute_error(X, y, j, alpha, b)

                alpha_i_old = alpha[i]
                alpha_j_old = alpha[j]

                L, H = self._compute_bounds(y, i, j, alpha)
                if L == H:
                    continue

                eta = 2 * self.kernel(X[i], X[j]) - self.kernel(X[i], X[i]) - self.kernel(X[j], X[j])
                if eta >= 0:
                    continue

                alpha[j] -= (y[j] * (E_i - E_j)) / eta
                alpha[j] = min(H, max(L, alpha[j]))

                if abs(alpha[j] - alpha_j_old) < 1e-5:
                    continue

                alpha[i] += y[i] * y[j] * (alpha_j_old - alpha[j])

                b1 = b - E_i - y[i] * (alpha[i] - alpha_i_old) * self.kernel(X[i], X[i]) - y[j] * (alpha[j] - alpha_j_old) * self.kernel(X[i], X[j])
                b2 = b - E_j - y[i] * (alpha[i] - alpha_i_old) * self.kernel(X[i], X[j]) - y[j] * (alpha[j] - alpha_j_old) * self.kernel(X[j], X[j])

                if 0 < alpha[i] < self.C:
                    b = b1
                elif 0 < alpha[j] < self.C:
                    b = b2
                else:
                    b = (b1 + b2) / 2

            if np.linalg.norm(alpha - alpha_prev) < 1e-5:
                break

        return (alpha, b)

    # Compute the error for a given sample
    def _compute_error(self, X, y, i, alpha, b):
        return np.dot(alpha * y, [self.kernel(X[i], X[j]) for j in range(len(X))]) + b - y[i]

    # Compute the bounds L and H for the optimization problem
    def _compute_bounds(self, y, i, j, alpha):
        if y[i] != y[j]:
            L = max(0, alpha[j] - alpha[i])
            H = min(self.C, self.C + alpha[j] - alpha[i])
        else:
            L = max(0, alpha[i] + alpha[j] - self.C)
            H = min(self.C, alpha[i] + alpha[j])
        return L, H

    # Predict the class labels for a given set of samples
    def predict(self, X):
        predictions = []
        for i in range(len(X)):
            class_scores = []
            for model in self.models:
                alpha, b = model
                decision_value = 0
                for j in range(len(self.X_train)):
                    if alpha[j] > 0:
                        decision_value += alpha[j] * self.y_train[j] * self.kernel(X[i], self.X_train[j])
                decision_value += b
                class_scores.append(decision_value)

            # Choose the class with the highest decision value
            predictions.append(self.classes[np.argmax(class_scores)])

        return np.array(predictions)

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Preprocess the data: Standardize it
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm = SVM(C=1, max_iter=1000)

# Fit the SVM model
svm.fit(X_train, y_train)

# Make predictions on the test data
predictions = svm.predict(X_test)

# Print the results
print("Predictions:", predictions)
print("Actual Labels:", y_test)

# Calculate accuracy
accuracy = np.sum(predictions == y_test) / len(y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


Predictions: [1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0]
Actual Labels: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy: 63.33%


Algorithm: SVM Classification (One-vs-Rest Strategy)

1. Input:
    - Training data: X_train (features), y_train (labels)
    - Test data: X_test (features)
    - Regularization parameter: C
    - Maximum iterations: max_iter

2. Initialization:
    - Create an SVM object with C and max_iter.
    - Identify unique class labels in y_train (self.classes).

3. Training:
    - For each class_label in self.classes:
        a. Create binary labels (y_binary): 1 for current class, -1 for others.
        b. Train a binary SVM model using _train_svm(X_train, y_binary).
        c. Store the trained model (alpha, b) in self.models.

4. Prediction:
    - For each data point in X_test:
        a. For each model in self.models:
            i. Calculate the decision value using the kernel function and learned parameters (alpha, b).
            ii. Store the decision value for the current class.
        b. Predict the class label with the highest decision value.

5. Output:
    - Predicted labels for X_test.

_train_svm(X, y):

1. Initialization:
    - alpha = 0 (for all training samples)
    - b = 0

2. Optimization loop (for max_iter iterations):
    a. Store previous alpha values (alpha_prev).
    b. For each training sample i:
        i. Randomly select another sample j (different from i).
        ii. Calculate errors E_i and E_j using _compute_error().
        iii. Store old alpha values (alpha_i_old, alpha_j_old).
        iv. Calculate bounds L and H using _compute_bounds().
        v. If L == H, skip to the next iteration.
        vi. Calculate learning rate eta.
        vii. If eta >= 0, skip to the next iteration.
        viii. Update alpha[j] using the update rule.
        ix. Clip alpha[j] within bounds L and H.
        x. If alpha[j] changed significantly, update alpha[i] and b.
    c. If alpha values converged (change is small), break the loop.

3. Output:
    - Learned parameters (alpha, b).

_compute_error(X, y, i, alpha, b):

1. Calculate the error for sample i using the decision function and learned parameters.

_compute_bounds(y, i, j, alpha):

1. Calculate the lower (L) and upper (H) bounds for alpha[j] based on the labels of samples i and j and the regularization parameter C.